## Установка необходимых библиотек и зависимостей

In [ ]:
!pip install --quiet pandas==1.5.3
!pip install --quiet jax>=0.1.72 tensorflow>=2.2.0 requests==2.27.1
!pip install --upgrade --quiet beautifulsoup4 selenium googletrans pyautogui opencv-python slugify huggingface_hub datasets pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 43.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 11.

## Парсинг изображений и текстовых подписей с веб-страницы
Парсинг веб-страницы с использованием библиотек requests и BeautifulSoup. Код извлекает ссылки на изображения и текстовые подписи с веб-страницы, используя заданную URL-адрес исходной страницы.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Ссылка на исходную страницу
url = 'https://www.houzz.com/photos/phbr31-bp~t_736~a_18-85-86-87--30-214-215-231--31-217-218-219-220--32-221-222-223-504--64-505-506-508-509-512-513-514-15728--69-548-549-550-551-553--3410-20895-20896-20897-20898?pg='

image_links_list = []
desc_texts_list = []

for i in range(270):
    # Отправляем GET-запрос к указанной странице
    page_url = url + f'{i+1}'
    while True:
        try:
            response = requests.get(page_url)
            break
        except (requests.exceptions.RequestException, IncompleteRead, ChunkedEncodingError) as e:
            print(f"Error occurred: {e}. Retrying in 5 seconds...")
            time.sleep(5)

    # Создаем объект BeautifulSoup для парсинга HTML-кода
    soup = BeautifulSoup(response.content, 'html.parser')

    # Находим все элементы <img> с классом "hz-photo-card__img"
    image_tags = soup.find_all('img', class_='hz-photo-card__img')
    desc_tags = soup.find_all('div', class_='hz-photo-card__desc-text text-xs hz-photo-card__no-peek')

    # Извлекаем ссылки на изображения и текстовые подписи
    image_links = [img['src'] for img in image_tags]
    desc_texts = [desc.get_text(strip=True) for desc in desc_tags]

    image_links_list += image_links
    desc_texts_list += desc_texts

# Создаем датафрейм
data = {
    'id': range(1, len(image_links_list) + 1),
    'img': image_links_list,
    'descr': desc_texts_list
}
df = pd.DataFrame(data)
len(df)

5400

## Отображение изображений и описаний из DataFrame
Вывод индекса и описания из каждой строки. Кроме того, код отображает соответствующее изображение с использованием ссылки из столбца 'img'.

In [ ]:
import IPython.display as display

# Итерация по каждой строке DataFrame
for index, row in new_df_out.head(10).iterrows():
    # Получение значений из столбцов 'img' и 'descr'
    image_link = row['img']
    description = row['descr']

    # Вывод индекса и описания
    print("Index:", index)
    print("Description:", description)

    # Отображение изображения
    display.display(display.Image(url=image_link))
    print("-------------------------------------")

Index: 30
Description: Photo: SG2 designInspiration for a mid-sized eclectic multicolored two-story brick exterior home remodel with a metal roof and a gray roof


-------------------------------------
Index: 32
Description: Inspiration for a large rustic brown two-story wood and board and batten exterior home remodel in Boise with a shingle roof and a brown roof


-------------------------------------
Index: 37
Description: Horizontal and vertical wood grid work wood boards  is overlaid on an existing 1970s home and act architectural layers to the interior of the home providing privacy and shade. A pallet of three colors  help to distinguish the layers. The project is the recipient of a National Award from the American Institute of Architects: Recognition for Small Projects.  !t also was one of three  houses designed by Donald Lococo Architects that received the  first place International HUE award for architectural color by Benjamin Moore


-------------------------------------
Index: 40
Description: We added garage on the left and new driveway and landscapingLarge french country red two-story brick exterior home photo in Other with a shingle roof and a gray roof


-------------------------------------
Index: 45
Description: Sunken family room addition with ICF insulated crawl space beneath it. Heated tile floors, the addition of a powder room, fireplace and large format sliding doors make this space comfortable and well connected to the exterior patio.


...

-------------------------------------
Index: 48
Description: Inspiration for a large contemporary gray split-level brick exterior home remodel in Newcastle - Maitland with a metal roof and a gray roof


-------------------------------------
Index: 53
Description: アウトドアを楽しむ 無垢の平屋の暮らし
～緑を感じておうちで過ごす～
木のぬくもりや、癒しの香りそして緑が心地よい空間を演出。深呼吸したくなる住まいです。
平屋ならではのフラットな動線が生み出す、ゆたかな暮らしを是非ご体感下さい。Small gray one-story mixed siding exterior home photo in Other with a metal roof and a black roof


-------------------------------------
Index: 55
Description: Mid-sized farmhouse white two-story mixed siding and board and batten exterior home idea in Other with a shingle roof and a gray roof


-------------------------------------
Index: 63
Description: Two story transitional style house in the desirable city of Winter Park, Florida (just north of Orlando). This white stucco house with contrasting gray tiled roof, black windows and aluminum shutters has instant curb appeal. Paver driveway and gas lit lanterns and sconces lead you through the landscaped front yard to the large pivot front door. Accent white brick adds texture and sophistication to the building's facade.


-------------------------------------


## Очистка текста и удаление повторяющихся строк в DataFrame
Очистка текста от нежелательных символов с помощью функции clean_text.

In [ ]:
import re

# Функция для очистки текста от нежелательных символов
def clean_text(text):
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s.,-:;]', '', text)
    return cleaned_text

# Создание копии DataFrame new_df
new_df_copy = df.copy()

# Очистка описаний в копии DataFrame
new_df_copy['descr'] = new_df_copy['descr'].apply(clean_text)

Удаление повторяющихся строк в DataFrame new_df_copy на основе столбцов 'img' и 'descr'.

In [ ]:
# Проверка и удаление повторяющихся строк по столбцам 'img' и 'descr'
new_df_copy.drop_duplicates(subset=['img', 'descr'], keep='first', inplace=True)

In [ ]:
len(new_df_copy)

3303

## Фильтрация DataFrame на основе наличия целевых слов
Фильтрация DataFrame new_df_copy на основе наличия целевых слов из списка target_words.

In [ ]:
# Список слов для фильтрации
target_words = ['exterior', 'home', 'house', 'compact', 'medium', 'large', 'one-story', 'two-story', 'three-story', 'story', 'split-level', 'split', 'level', 'siding material', 'mixed', 'wood', 'stucco', 'brick', 'white', 'beige', 'gray', 'brown', 'multi', 'black', 'red', 'yellow', 'gable', 'hip', 'shed', 'flat', 'clipped']

# Фильтрация датафрейма на основе наличия целевых слов
filtered_df_tw6 = new_df_copy[new_df_copy['descr'].apply(lambda x: sum(word in x for word in target_words) >= 6)]
filtered_df_tw7 = new_df_copy[new_df_copy['descr'].apply(lambda x: sum(word in x for word in target_words) >= 7)]
filtered_df_tw8 = new_df_copy[new_df_copy['descr'].apply(lambda x: sum(word in x for word in target_words) >= 8)]

In [ ]:
len(filtered_df_tw6), len(filtered_df_tw7), len(filtered_df_tw8)

(2344, 2140, 1351)

# Чтение экселевского файла с Google Диска и отображение изображений
Чтение экселевского файла с Google Диска, проверка отсутствующих изображений, добавление отсутствующих строк в конец экселевского файла.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Чтение экселевского файла с гугл диска
csv_file_path = '/content/drive/MyDrive/Colab Notebooks/Study/DrawHomeAI/Subsets/filtered_df_tw6 - filtered_df_tw6.csv'
csv_df = pd.read_csv(csv_file_path)

# Проверка отсутствующих изображений
missing_images = filtered_df_tw6[~filtered_df_tw6['img'].isin(csv_df['img'])]

# Добавление отсутствующих строк в конец экселевского файла
updated_csv_df = pd.concat([csv_df, missing_images])

# Сохранение обновленного экселевского файла
updated_csv_file_path = '/content/drive/MyDrive/Colab Notebooks/Study/DrawHomeAI/Subsets/updated_filtered_df_tw6.csv'
updated_csv_df.to_csv(updated_csv_file_path, index=False)

Отображение изображений с описаниями.

In [ ]:
import IPython.display as display

# Итерация по каждой строке DataFrame
for index, row in filtered_df_tw6.head(10).iterrows():
    # Получение значений из столбцов 'img' и 'descr'
    image_link = row['img']
    description = row['descr']

    # Вывод индекса и описания
    print("Index:", index)
    print("Description:", description)

    # Отображение изображения
    display.display(display.Image(url=image_link))
    print("-------------------------------------")

Index: 1
Description: Inspiration for a large cottage white two-story wood exterior home remodel in Denver with a mixed material roof and a red roof


-------------------------------------
Index: 5
Description: Tripp SmithLarge beach style brown three-story wood and shingle house exterior photo in Charleston with a hip roof, a mixed material roof and a gray roof


-------------------------------------
Index: 6
Description: Rear garden and a full length extension with folding glass doors.

 Photo by Chris SnookExample of a large minimalist multicolored three-story stucco exterior home design in London with a shingle roof and a red roof


-------------------------------------
Index: 8
Description: Large contemporary multicolored three-story brick exterior home idea in San Francisco with a metal roof and a black roof


-------------------------------------
Index: 9
Description: Large elegant red three-story brick exterior home photo in Chicago with a shingle roof and a gray roof


...

-------------------------------------
Index: 12
Description: Example of a mid-sized arts and crafts gray two-story wood and shingle exterior home design in Other with a shingle roof and a gray roof


-------------------------------------
Index: 13
Description: Kiawah Island Real EstateMid-sized transitional gray three-story wood exterior home photo in Charleston with a shingle roof and a gray roof


-------------------------------------
Index: 14
Description: View from beach.Inspiration for a mid-sized coastal gray two-story mixed siding and board and batten exterior home remodel in Seattle with a shingle roof and a gray roof


-------------------------------------
Index: 15
Description: Example of a large danish white two-story mixed siding house exterior design in Other with a shed roof, a metal roof and a gray roof


-------------------------------------
